# Task 1 - Exploratory Data Analysis

## Problem and Task
You’ve landed a great job with the Ritz-Jager telecommunication operator, as a data scientist.
<u>This telecommunication operator wants to improve their business efficiency and know their
customers better by utilizing their customer data and they want to find out about several business
questions from the company’s stakeholder</u>. Your team of engineer have to analyze the data that
they have based on the pre-defined questions that your CEO gave.

## Student workflow:
- Collecting the data
- Transforming the data
- Analyzing the data
- Answering the business questions by creating the data
- Visualization


In [ ]:
!pip install -U -q fancyimpute

In [ ]:
# import library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import os
import fancyimpute
import warnings
warnings.filterwarnings("ignore")

## Dataset Description
Dataset memiliki jumlah `7043` observations dengan `21` features dan 1 label (`Churn`)

| __Feature Name__ | __Description__ |
| - | - |
| customerID | Contains customer ID |  
| gender | whether the customer female or male | 
| SeniorCitizen | Whether the customer is a senior citizen or not (1, 0) | 
| Partner | Whether the customer has a partner or not (Yes, No) | 
| Dependents | Whether the customer has dependents or not (Yes, No) |  
| tenure | Number of months the customer has stayed with the company | 
| PhoneService | Whether the customer has a phone service or not (Yes, No) | 
| MultipleLines | Whether the customer has multiple lines r not (Yes, No, No phone service) |
| InternetService | Customer’s internet service provider (DSL, Fiber optic, No) | 
| OnlineSecurity | Whether the customer has online security or not (Yes, No, No internet service) | 
| OnlineBackup |  Whether the customer has online backup or not (Yes, No, No internet service) | 
| DeviceProtection | Whether the customer has device protection or not (Yes, No, No internet service) | 
| TechSupport | Whether the customer has tech support or not (Yes, No, No internet service) | 
| streamingTV | Whether the customer has streaming TV or not (Yes, No, No internet service) | 
| streamingMovies | Whether the customer has streaming movies or not (Yes, No, No internet service) | 
| Contract | The contract term of the customer (Month-to-month, One year, Two year) | 
| PaperlessBilling | Whether the customer has paperless billing or not (Yes, No) | 
| PaymentMethod | The customer’s payment method (Electronic check, Mailed check, Bank transfer, Credit card) | 
| MonthlyCharges | The amount charged to the customer monthly  |  
| TotalCharges | The total amount charged to the customer  | 
| Churn | Whether the customer churned or not (Yes or No) |

## Collect and load data
Load dataset, data yang disediakan Iykra banyak terdapat kolom kosong dibandingkan dengan data source asli.

In [ ]:
df = pd.read_csv('../input/telcocustchurniykra/data.csv')
df.head()

In [ ]:
# df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
# df.head()

## Statistical Summary
Hanya 3 data yang bertipe numerik pada DataFrame. Perlu dilakukan cek tipe data pada tiap atribut apakah sudah tepat tipe datanya.

In [ ]:
df.describe()

## Data Cleansing
Kita terlebih dahulu cek apakah data terdapat nilai kosong, jenis tipe data, jumlah data, atribut data.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

Jumlah data lengkap: **7043 data**

Data yang tersedia terdapat NULL sejumlah **1009** pada atribut:
- `gender`, `SeniorCitizen`, `Partner`, `Dependents`, `MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`

> Sebelum melakukan imputasi untuk mengisi data kosong, kita coba lihat value unik pada tiap atribut.

In [ ]:
for col in df.columns:
    print("%s unique count: %d" % (col, df[col].nunique()))
    print(df[col].unique(), '\n')

Terdapat dua jenis data disini:

- **Categorical data**: Data berupa kategori, misalkan pada atribut `gender`

- **Numerical data**: Data berupa angka, misalkan pada atribut `MonthlyCharges`

Jika diperhatikan, masih terdapat kesalahan jenis tipe data seperti atribut `TotalCharges` yang bertipe object, padahal seharusnya bertipe numerik `float`. Maka harus dibenarkan terlebih dahulu.

Berdasarkan info data unik tersebut kita bisa tentukan tipe data yang benar, berikut pembagian tipe data:
- **Categorical data**

|   | Attribute       |    | Attribute          |    | Attribute          |    | Attribute       |
|---|-----------------|----|--------------------|----|--------------------|----|-----------------|
| 1 | `gender`        | 6  | `MultipleLines`    | 11 | `TechSupport`      | 16 | `PaymentMethod` |
| 2 | `SeniorCitizen` | 7  | `InternetService`  | 12 | `StreamingTV`      | 17 | `Churn`         |
| 3 | `Partner`       | 8  | `Online Security`  | 13 | `StreamingMovies`  |    |                 |
| 4 | `Dependents`    | 9  | `Online Backup`    | 14 | `Contract`         |    |                 |
| 5 | `PhoneService`  | 10 | `DeviceProtection` | 15 | `PaperlessBilling` |    |                 |

- **Numerical data**

|   | Attribute                |
|---|--------------------------|
| 1 | `tenure` (int)           |
| 2 | `MonthlyCharges` (float) |
| 3 | `TotalCharges` (float)   |

Sedangkan untuk atribut `customerID` tidak kita gunakan dalam analisis data karena tidak berisi informasi penting untuk menjawab pertanyaan bisnis.

In [ ]:
# drop customerID
df = df.drop(['customerID'], axis=1)

### Mapping data (Encoding)
Sebelum melakukan impute data kosong, kita coba lakukan mapping data string menjadi numerik.

Strategi:
- Replace redundan nilai seperti 'No phone service' dan 'No internet service' menjadi 'No'
- Replace categorical data yang memiliki nilai 'Yes' menjadi 1 dan 'No' menjadi 0
- Replace 'Male' menjadi 1 dan 'Female' menjadi 0
- Pada atribut `InternetService`
    * Replace 'DSL' menjadi 1 
    * Replace 'Fiber optic' menjadi 2
    * Replace 'No' menjadi 3
- Pada atribut `Contract`
    * Replace 'Month-to-month' menjadi 1
    * Replace 'One year' menjadi 2
    * Replace 'Two year' menjadi 3
- Pada atribut `PaymentMethod`
    * Replace 'Electronic check' menjadi 1
    * Replace 'Mailed check' menjadi 2
    * Replace 'Bank transfer (automatic)' menjadi 3
    * Replace 'Credit card (automatic)' menjadi 4
    
> Note: Tidak menggunakan One Hot Encoding pada kategori lebih dari 2, karena untuk kepentingan impute data menggunakan KNN. Alternatif lain selain manual mapping, bisa menggunakan OrdinalEncoder dari scikit-learn.

In [ ]:
# replace 'No phone service' menjadi 'No' pada MultipleLines
df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 'No'})

# replace 'No internet service' menjadi 'No'
cols_redundant = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                  'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in cols_redundant:
    df[col] = df[col].replace({'No internet service': 'No'})

In [ ]:
cols_cat = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity',
            'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
            'PaperlessBilling', 'Churn']

# replace 'Yes' menjadi 1 dan 'No' menjadi 0
for col in cols_cat:
    df[col] = df[col].replace({'Yes': 1, 'No': 0})

In [ ]:
# replace 'female' menjadi 1 dan 'male' menjadi 0
df['gender'] = df['gender'].replace({'Male': 1, 'Female': 0})

In [ ]:
# Pada atribut InternetService
## Replace 'DSL' menjadi 1
## Replace 'Fiber optic' menjadi 2
## Replace 'No' menjadi 3
df['InternetService'] = df['InternetService'].replace({'DSL': 1,
                                                       'Fiber optic': 2,
                                                       'No': 3})

In [ ]:
# Pada atribut Contract
##abs Replace 'Month-to-month' menjadi 1
## Replace 'One year' menjadi 2
## Replace 'Two year' menjadi 3
df['Contract'] = df['Contract'].replace({'Month-to-month': 1,
                                         'One year': 2,
                                         'Two year': 3})

In [ ]:
# Pada atribut PaymentMethod
## Replace 'Electronic check' menjadi 1
## Replace 'Mailed check' menjadi 2
## Replace 'Bank transfer (automatic)' menjadi 3
## Replace 'Credit card (automatic)' menjadi 4
df['PaymentMethod'] = df['PaymentMethod'].replace({'Electronic check': 1,
                                                   'Mailed check': 2,
                                                   'Bank transfer (automatic)': 3,
                                                   'Credit card (automatic)': 4})

### Fixing atribut TotalCharges


In [ ]:
# df['TotalCharges'] = df['TotalCharges'].astype('float64')

Ketika merubah tipe data pada `TotalCharges` mengalami error. Sebabnya adalah terdapat nilai kosong, yang tidak terdeteksi ketika melakukan pengecekan menggunakan `isnull()`.

In [ ]:
df.TotalCharges.isnull().sum()

In [ ]:
# cek data berisi spasi pada TotalCharges
temp = df.sort_values('TotalCharges') 
tcharges = temp.TotalCharges[temp['TotalCharges'] == ' ']
print("Jumlah data kosong: ", len(tcharges))
tcharges

In [ ]:
# terdapat karakter spasi pada data
temp = df.sort_values('TotalCharges')['TotalCharges'][:15][936] 
temp, len(temp)

Walaupun data yang kosong berjumlah 11 data, kita boleh saja menghilangkannya karena hanya 0.15% dari data.
Tetapi, kita bisa mengisinya dengan mengalikan `tenure` dengan `MonthlyCharges`. Karena atribut `tenure` adalah jumlah bulan ketika customer tetap menggunakan layanan perusahaan, sedangkan atribute `MonthlyCharges` adalah jumlah yang harus dibayar tiap bulan.

Sehingga formula untuk mengisi data kosong pada `TotalCharges` adalah

$TotalCharges = tenure * MonthlyCharges$

In [ ]:
# convert space menjadi NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# setelah dirubah ke NaN, data kosong bisa diketahui dengan fungsi isnull()
print("Data kosong pada TotalCharges:", df.TotalCharges.isnull().sum())

# isi data kosong pada TotalCharges dengan formula di atas
df['TotalCharges'].fillna(value=df['tenure'] * df['MonthlyCharges'], inplace=True)
print("Data kosong pada TotalCharges:", df.TotalCharges.isnull().sum()) # cek lagi data kosong

In [ ]:
df.info()

### Impute categorical data
Data yang berisi kosong:
-  `gender`, `SeniorCitizen`, `Partner`, `Dependents`, `MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`

Strategi:
- Karena data yang NULL adalah categorical data, maka dilakukan impute data kosong menggunakan KNN

In [ ]:
# initialize
imputer = fancyimpute.KNN()

# impute data and convert 
encode_data = pd.DataFrame(np.round(imputer.fit_transform(df)),columns = df.columns)

Hasil dari imputasi dengan KNN, coba kita cek komposisi data dengan data sebelumnya. Seharusnya tidak jauh berbeda.

In [ ]:
cols_impute = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines', 
               'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
               'StreamingTV', 'StreamingMovies']

for col in cols_impute:
    data_before = df[col].value_counts(normalize=True)
    data_after = encode_data[col].value_counts(normalize=True)
    
    combine_data = pd.DataFrame()
    combine_data['percentage_before'] = data_before
    combine_data['percentage_after'] = data_after
    print(col)
    print(combine_data)
    print()

In [ ]:
# cek lagi null data
encode_data.isnull().any().sum()

### Assign tipe data
Selanjutnya, assign tipe data yang benar untuk tiap atribut menjadi:
- int64
    * `gender`, `SeniorCitizen`, `Partner`, `Dependents`, `tenure`, `PhoneService`, `MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`, `Contract`, `PaperlessBilling`, `PaymentMethod`, `Churn`
- float64
    * `MonthlyCharges`, `TotalCharges`

In [ ]:
cols_float = ['MonthlyCharges', 'TotalCharges']
cols_int = [x for x in list(encode_data.columns) if not x in cols_float]

encode_data[cols_float] = encode_data[cols_float].astype('float64')
encode_data[cols_int] = encode_data[cols_int].astype('int64')
encode_data.info()

## Data Exploratory Analysis
Setelah kita melakukan validasi data, kita dapat melakukan *explore* dan analisis untuk mendapatkan insight.

> Untuk kepentingan visualisasi, maka replace kembali data numerik menjadi string.

In [ ]:
df_viz = encode_data.copy()

In [ ]:
cols_cat = ['SeniorCitizen','Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
            'OnlineSecurity','OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 
            'StreamingMovies','PaperlessBilling', 'Churn']

for col in cols_cat:
    df_viz[col] = df_viz[col].replace({1:'Yes', 0:'No'})

# gender
df_viz['gender'] = df_viz['gender'].replace({1:'Male', 0:'Female'})

# InternetService
df_viz['InternetService'] = df_viz['InternetService'].replace({1:'DSL',
                                                               2:'Fiber optic',
                                                               3:'No'})
# Contract
df_viz['Contract'] = df_viz['Contract'].replace({1:'Month-to-month',
                                                 2:'One year',
                                                 3:'Two year'})
# PaymentMethod
df_viz['PaymentMethod'] = df_viz['PaymentMethod'].replace({1:'Electronic check',
                                                           2:'Mailed check',
                                                           3:'Bank transfer (automatic)',
                                                           4:'Credit card (automatic)'})

In [ ]:
df_viz.head()

### Explore numerical variable
Kita lakukan observasi dari statistik data pada atribut:
- `tenure`: Number of months the customer has stayed with the company
- `MonthlyCharge`: The amount charged to the customer monthly
- `TotalCharges`: The total amount charged to the customer

In [ ]:
# statistical summary
df_viz.describe().round(3)

Terdapat beberapa insight yang kita ketahui:
- Berdasarkan median `tenure`, kita mengetahui bahwa 50% dari customer tetap di perusahaan selama 29 bulan
- Rata-rata tagihan bulanan customer adalah \$65
- Rata-rata pendapatan yang didapatkan perusahaan dari tiap customer adalah \$2280

Selanjutnya, kita coba lihat hubungan antar variabel dengan melakukan plotting

In [ ]:
df_viz[df_viz['Churn'] == 'Yes']['MonthlyCharges'].mean()

In [ ]:
df_viz[df_viz['Churn'] == 'No']['TotalCharges'].mean()

In [ ]:
df_num = df_viz[['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']]

### tenure vs TotalCharges

In [ ]:
plt.figure(figsize=(12,8))
sns.lmplot(x='tenure', y='TotalCharges', col='Churn', hue='Churn', data=df_num)
# plt.title('tenure vs TotalCharges')

### MonthlyCharges vs TotalCharges

In [ ]:
plt.figure(figsize=(12,8))
sns.lmplot(x='MonthlyCharges', y='TotalCharges', col='Churn', hue='Churn', data=df_num)
# plt.title('MonthlyCharges vs TotalCharges')

### tenure vs MonthlyCharges

In [ ]:
plt.figure(figsize=(12,8))
sns.lmplot(x='tenure', y='MonthlyCharges', col='Churn', hue='Churn', data=df_num)
# plt.title('MonthlyCharges vs TotalCharges')

### Pairplot data numeric

In [ ]:
plt.figure(figsize = (15,10))
sns.pairplot(df_num, hue='Churn')

### Boxplot tenure vs TotalCharges

In [ ]:
plt.figure(figsize=(17,10))

sns.boxplot(x='tenure', y='TotalCharges', data=df_num)

plt.title('Box Plot of Tenure X TotalCharges')

### Countplot tenure

In [ ]:
plt.figure(figsize = (17,10))
sns.countplot(df_viz['tenure'])

### Analisis data numeric
- Pada plot `tenure vs TotalCharges`, sesuai dengan dugaan kita, bahwa semakin lama customer berada di perusahaan (tenure makin besar), maka semakin besar juga total tagihan customer.
- Pada plot `MonthlyCharges vs TotalCharges`, dapat terlihat juga semakin besar tagihan bulanan customer, maka semakin besar juga total tagihannya.
- Pada plot `tenure vs MonthlyCharges`, tidak terdapat korelasi keduanya. Artinya, terdapat customer yang menggunakan service yang sama tagihan bulanannya dalam jangka panjang dan terdapat customer yang langsung menggunakan service dengan tagihan yang langsung mahal.
- Pada `Box Plot`, dapat kita lihat tidak terdapat data outlier.
- Pada `barchart tenure count`, dapat kita ketahui bahwa banyak customer yang hanya menggunakan service selama 1 bulan. Tetapi, pada peringkat terbanyak ke-2 dapat kita lihat bahwa banyak customer yang masih bertahan menggunakan service dari perusahaan selama 72 bulan.  

### Explore categorical variable

In [ ]:
df_viz.describe(include='object')

Kita dapat melihat beberapa atribut statistik, seperti:
- Customer didominasi oleh bukan senior pada `SeniorCitizen`
- Layanan fiber optic yang paling populer pada `InternetService`
- Sebagian besar `Contract` customer adalah Month-to-month
- `PaymentMethod` customer sebagian besar menggunakan Electronic check

Untuk dapat melihat perbandingan kategori, dapat kita plotting menggunakan pie chart.

In [ ]:
# pie chart
cols_cat = df_viz.describe(include='object').columns

fig = plt.figure(figsize=(16,35), dpi=80)
plt.subplots_adjust(hspace=.35)

for idx, item in enumerate(cols_cat, 1):
    data = df_viz[item].value_counts().to_frame().T
    labels = data.columns
    title = df_viz[item].name
    
    fig.add_subplot(6,3,idx)
    plt.title(title, fontsize=14)
    plt.pie(data, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 14})
    plt.axis('equal')

Karena kita berfokus pada churn customer, maka coba kita explore atribute apa yang berperan besar dalam churn.

### Pengelompokkan data categorical
Berdasarkan atribut yang tersedia, dapat kita kelompokkan menjadi sebagai berikut:
- Services (9 atribut)
    * `PhoneService`,`MultipleLines`,`InternetService`,`OnlineSecurity`, `OnlineBackup`,      `DeviceProtection`,`TechSupport`,`StreamingTV`,`StreamingMovies`
- Customer info (4 atribut)
    * `gender`, `SeniorCitizen`, `Partner`, `Dependents`
- Other (3 atribut)
    * `Contract`, `PaperlessBilling`, `PaymentMethod`


Sedangkan `Churn` adalah target yang akan kita bandingkan dengan atribut lainnya.

### Churn terhadap kelompok Services

In [ ]:
cat_services = ['PhoneService','MultipleLines','InternetService',
                'OnlineSecurity', 'OnlineBackup','DeviceProtection',
                'TechSupport','StreamingTV','StreamingMovies']

fig = plt.figure(figsize=(18,18), dpi=80)
plt.subplots_adjust(hspace=.35)

for idx, col in enumerate(cat_services, 1):
    fig.add_subplot(3,3,idx)
    plt.title(f'{col} Counts', fontsize=16)
    countplot_sc = sns.countplot(df_viz[col], hue=df_viz.Churn)
    
    # text annotation
    for p in countplot_sc.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        countplot_sc.annotate('{:.2f}%'.format(100.*y/len(df_viz[col])), 
                              (x.mean(), y),
                              ha='center',
                              va='bottom') # set the alignment of the text

> `InternetService` menggunakan Fiber Optic berdampak Churn dibandingkan yang lain.

### Churn terhadap kelompok Customer info

In [ ]:
cat_custinfo = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']

fig = plt.figure(figsize=(12,10), dpi=80)
plt.subplots_adjust(hspace=.35)

for idx, col in enumerate(cat_custinfo, 1):
    fig.add_subplot(2,2,idx)
    plt.title(f'{col} Counts', fontsize=16)
    countplot_sc = sns.countplot(df_viz[col], hue=df_viz.Churn)
    
    # text annotation
    for p in countplot_sc.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        countplot_sc.annotate('{:.2f}%'.format(100.*y/len(df_viz[col])), 
                              (x.mean(), y),
                              ha='center',
                              va='bottom') # set the alignment of the text

> `SeniorCitizen` berdampak pada Churn dibandingkan yang lain

### Churn terhadap kelompok Other

In [ ]:
cat_other = ['Contract', 'PaperlessBilling', 'PaymentMethod']

fig = plt.figure(figsize=(12,10), dpi=80)
plt.subplots_adjust(hspace=.35)

for idx, col in enumerate(cat_other, 1):
    fig.add_subplot(2,2,idx)
    plt.title(f'{col} Counts', fontsize=16)
    countplot_sc = sns.countplot(df_viz[col], hue=df_viz.Churn)
    
    if idx == 3:
        countplot_sc.set_xticklabels(countplot_sc.get_xticklabels(), rotation=20)
    
    # text annotation
    for p in countplot_sc.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        countplot_sc.annotate('{:.2f}%'.format(100.*y/len(df_viz[col])), 
                              (x.mean(), y),
                              ha='center',
                              va='bottom') # set the alignment of the text

> `Contract` dengan Month-to-month berdampak pada Churn

> `PaymentMethod` dengan Electronic check berdampak pada Churn 

## Finding Correlation

In [ ]:
data = df_viz.copy()
data.head()

### Encoding
- Binary encoding pada atribut yang memiliki 2 kategori
- One Hot Encoding pada atribut yang memiliki lebih dari 2 kategori

In [ ]:
# binary encoding
gender_map = {"Female" : 0, "Male": 1}
yes_no_map = {"Yes" : 1, "No" : 0}

data["gender"] = data["gender"].map(gender_map)

binary_encode_cols = ['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                      'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 
                      'StreamingMovies', 'PaperlessBilling', 'Churn']

for col in binary_encode_cols:
    data[col] = data[col].map(yes_no_map)

In [ ]:
data.head()

In [ ]:
# one hot encoding
onehot_encode_cols = ['InternetService', 'Contract', 'PaymentMethod']
data_onehot_encode = pd.get_dummies(data[onehot_encode_cols])

# join
data = data.drop(onehot_encode_cols, axis=1)
data = pd.concat([data, data_onehot_encode], axis=1)

In [ ]:
# change data type from category to uint8
for col in data.columns:
    if str(data[col].dtypes) == 'category':
        data[col] = data[col].astype('uint8')

In [ ]:
plt.figure(figsize = (15, 15))
sns.heatmap(data.corr(), cmap="RdYlBu", annot=True, fmt=".1f")
plt.show()

In [ ]:
data.corr()['Churn'].sort_values(ascending=False)

> Top 3 fitur yang berkorelasi positif terhadap `Churn` adalah `Contract_Month-to-month`, `InternetService_Fiber optic`, dan `PaymentMethod_Electronic check`

> Top 3 fitur yang berkorelasi negative terhadap `Churn` adalah `tenure`, `Contract_Two year`, dan `InternetService_No`